In [ ]:
import pandas as pd
import numpy as np
import pickle

seed = 42
np.random.seed(seed)
lig_num = 10
sample_size = 1000
estimators = 100

In [ ]:
with open("../models/tfidf.pickle", "rb") as fp:
    tfidf = pickle.load(fp)

In [ ]:
df_sequences = pd.read_csv("sequences.csv", index_col="index")
df_binding = pd.read_csv("lig2seq.csv")

lig_id_vals = list(np.random.sample(df_binding["lig"].unique(), size=lig_num))
df_binding = df_binding.loc[df_binding["lig"].isin(lig_ids_vals)]
df_binding["seq"] = df_binding["seq"].apply(lambda x: df_sequences.loc[x, "sequence"])

In [ ]:
from utils.baggingPU import BaggingClassifierPU
from sklearn.tree import DecisionTreeClassifier


models = {}
for i, lig_id in enumerate(lig_id_vals):
    seq_id = list(df_binding.loc[df_binding["lig"]==lig_id, "seq"])
    if len(seq_id) < sample_size//2 and len(seq_id) > 0:
        print(len(seq_id))
        df_seq_sub_neg = df_sequences.loc[~df_sequences.index.isin(seq_id)].sample(n=sample_size-len(seq_id), random_state=seed+i)
        df_seq_sub_neg["bind"] = 0
        df_seq_sub_neg["bind_t"] = 0
        df_seq_sub_pos = df_sequences.loc[df_sequences.index.isin(seq_id)].sample(frac=0.75, random_state=seed+i)
        df_seq_sub_pos["bind"] = 1
        df_seq_sub_pos["bind_t"] = 1
        df_seq_sub_pos_hid = df_sequences.loc[(df_sequences.index.isin(seq_id))&&(~df_sequences.index.isin(df_seq_sub_pos.index))]
        df_seq_sub_pos_hid["bind"] = 0
        df_seq_sub_pos_hid["bind_t"] = 1
        
        df_fitter = pd.concat([df_seq_sub_neg, df_seq_sub_pos, df_seq_sub_pos_hid])
        df_fitter["sequence"] = df_fitter["sequence"].apply(tfidf.transform([x]))
        results[["truth", "label"]] = df_fitter[["bind", "bind_t"]]
        
        X = np.array([df_fitter["sequence"].values[i] for i in range(df_fitter["sequence"].values.shape[0])])
        y = df_fitter["bind"]
        
        bc = BaggingClassifierPU(DecisionTreeClassifier(), n_estimators=estimators, n_jobs=-1, max_samples=sum(y))
        bc.fit(X, y)
        models[lig_id] = pickle.dumps(bc)

In [ ]:
with open("../models/PUC.pickle", "wb") as fp:
    pickle.dump(models, fp)